EDA

- 별점 낮은 리뷰에서는 어떤 요소가 불호 워드로 나왔는지 → 개선점
- 리뷰수가 많을수록 평균 별점이 높았을거다
- 광고모델에따른 인기도가 다를 것이다
- 브랜드별 선호도나 제품종류별 특징
- 선크림, 선쿠션, 선스틱, 종류별 어떤 키워드가 많이 나왔는지

In [1]:
!pip install konlpy
from konlpy.tag import Komoran
komoran = Komoran()

  Using cached konlpy-0.6.0-py2.py3-none-any.whl.metadata (1.9 kB)
Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)


In [2]:
import pandas as pd
import re
from tqdm.notebook import tqdm

In [4]:
# analysis_pipeline.py

import re
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1) 데이터 불러오기
df = pd.read_csv('C:/Users/LG/OneDrive - 숙명여자대학교/텍스트마이닝과자연어분석/crawling_final.csv', encoding="cp949")
#2) 이모티콘 제거
def clean_text(s):
    # 한글·영문·숫자·공백만 남기기
    return re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', str(s))
df['cleaned'] = df['review'].apply(clean_text)

df['cleaned']

0                    수분크림처럼 부드럽게 피부에 스며들어 촉촉히게 발리는 라네즈 썬크림
1                크림같아서 좋네요선크림 특유의 질감  냄새도 없고백탁도없고 그냥 크림같아요
2           수분크림을 바르는 듯한 발림성에 촉촉함까지   오랜만에 정말 만족스러운 썬크림이에요
3        백탁도 없고 발림성  흡수력 너무 좋아요  부담없는 선크림  수분크림 같아서 아주 ...
4        아 원지 역시 완벽주의자 원지가 만든 건 다 좋아 완전 로션제형에 슥슥 잘 발리고 ...
                               ...                        
34705            가족선물로 시켰어요 순하고 트러블 없고 톤업돼서  이 선크림만 사용중입니다
34706       몇년째 사용중인 선크림이에요 자극없이순하고 발림성도 좋고 톤업돼서 하루종일 화사해요
34707                이거 너무너무좋아요   톤업도 잘되고 결정적으로 색상이 너무 이뻐용
34708    촉촉한데 바르면 보송하게 마무리 됩니다 야외 활동이 많아진 무더운 여름몸 전체에 바...
34709    백탁없다고 해도 은근히 백탁있는 제품이 많던데이건 진짜 백탁없어요 일주일동안 사용했...
Name: cleaned, Length: 34710, dtype: object

In [5]:
pip install jpype1-1.5.2-cp310-cp310-win_amd64.whl

Note: you may need to restart the kernel to use updated packages.


ERROR: jpype1-1.5.2-cp310-cp310-win_amd64.whl is not a supported wheel on this platform.


In [6]:
outword = ["때", "선크림", "썬크림", "선쿠션", '쌀', '요', '양', '형', '혼', '넘', '톤', '몰', '하', '연', '안', '폰', '티', '목', '립', '킴',
         '종', '전', '본', '가', '미', '다', '게', '아', '혀', '눈', '향', '함', '침', '맘', '후', '맥', '힘', '달', '겉', '선', '열', '땀',
         '날', '말', '밖', '여', '갈', '을', '못', '손', '물', '사', '폼', '입', '이', '당', '족', '파', '뒤', '볼', '코', '끔', '한', '톰',
         '업', '용', '콩', '떡', '허', '팁', '기', '뼈', '임', '끝', '감', '빛', '덧', '싱', '노', '칠', '품', '얼', '오', '위', '일', '틈',
         '리', '습', '속', '도', '뜸', '색', '어', '짜', '피', '발', '영', '남', '쌍', '야', '엉', '앞', '욥', '개', '귀', '썬', '골', '봄',
         '옆', '보', '팔', '인', '꿀', '그', '학', '괘', '은', '병', '해', '졸', '똥', '씀', '정', '두', '딸', '핑', '특', '알', '조', '흠',
         '팥', '합', '막', '잘', '난', '콘', '셀', '통', '누', '무', '더', '과', '팩', '화', '짐', '재', '지', '옷', '린', '모', '광', '글',
         '독', '애', '디', '션', '내', '매', '엽', '떼', '쌈', '뽕', '집', '능', '몸', '소', '추', '친', '완', '첨', '중', '곳', '쥬', '각',
         '느', '잇', '베', '챙', '겨', '서', '덤', '생', '마', '예', '세', '으', '염', '유', '나', '적', '러', '약', '새', '항', '상', '흰',
         '끼', '메', '컵', '악', '흑', '데', '음', '옴', '담', '획', '신', '로', '갑', '면', '단', '읍', '고', '곱', '띠', '엿', '넌', '냉',
         '백', '결', '존', '극', '총', '납', '라', '레', '래', '멜', '호', '턱', '타', '쿠', '뭐', '맛', '찬', '따', '녀', '살', '뺨', '뜰',
         '체', '절', '득', '드', '김', '규', '별', '변', '효', '운', '뭍', '길', '올', '죽', '순', '짝', '버', '릭', '겹', '끈', '냐', "요조",
         '편', '만', '밥', '식', '믈', '주', '풀', '갸', '부', '공', '혹', '력', '춤', '표', '배', '시', '밑', '낮', '진', '벗', '덕', '땅',
         '건', '릴', '돌', '착', '컷', '장', '비', '현', '불', '밤', '반', '니', '닷', '직', '처', '참', '벤', '박', '방', '국', '환', '쥐',
         '르', '룩', '암', '탓', '님', '홈', '패', '답', '차', '급', '점', '꿈', '에', '투', '숨', '강', '꽃', '랑', '까', '징', '걸', '센',
         '언', '왕', '엄', '찌', '링', '빠', '껌', '토', '카', '치', '질', '켄', '왜', '뜻', '구', '성', '좌', '우', '충', '거', '태', '증',
         '의', '저', '톱', '탁', '앎', '테', '섶', '림', '꼴', '팀', '액', '둑', '런', '탑', '룸', '싹', '락', '흉', '샘', '응', '송', '망',
         '홀', '줄', '낫', '필', '빙', '팝', '빅', '작', '문', '민', '실', '심', '텐', '죤', '울', '키', '값', '릉', '철', '동', '금', '팬',
         '홍', '창', '벡', '검', '깐', '칸', '휘', '율', '깃', '랩', '잎', '평', '숲', '논', '콜', '틱', '계', '류', '최', '축', '닐', '탄',
         '힐', '윤', '산', '므', '잠', '헌', '짓', '곰', '택', '낭', '덫', '몽', '역', '란', '들', '샹', '뇨', '핫', '범', '댐', '칩', '록',
         '외', '술', '빵', '궁', '뻘', '폴', '묘', '셤', '녜', '줌', '농', '샷', '삼', '찜', '와', '수', '먹', '펙', '초', '원', '폭', '혁',
         '믹', '법', '끌', '핵', '온', '닻', '청', '멱', '펄', '뤼', '량', '복', '잡', '묵', '칙', '솔', '맨', '잔', '컴', '자', '휴', '밭',
         '좀', '삽', '근', '접', '포', '썰', '땡', '탕', '붓', '펜', '탈', '벨', '솜', '댁', '램', '롤', '봉', '몬', '벽', '틀', '햄', '벼',
         '꾀', '멋', '씨', '껀', '젤', '즙', '활', '석', '꽈', '몫', '뱃', '옥', '녹', '촉', '킥', '엔', '볕', '멀', '쇼', '첩', '터', '겁', 
         '젓', '굴', '행', '붕', '훔', '루', '놀', '론', '깨', '솥', '경', '출', '갭', '욕', '갱', '밴', '널', '털', '돗', '쾌', '죄', '붐',
         '선스틱', '썬스틱', '구매', '제품', '사용', '느낌', '생각', '시나', '모로', "개도", "데바르", "이면", "이건", "아악", "전이", "호가", "안보",
          "군가", "지나", "리한", "여물", "바쿠", "바르", "쓰기", "하주", "이랑"]

In [8]:
tmp = []

for sentence in tqdm(df["cleaned"]):
    try:
        tokens = komoran.pos(sentence)
        nouns = [x[0] for x in tokens if x[1] in ("NNG", "NNP") and x[0] not in outword]
        tmp.append(nouns)
    except:
        print(f"Error processing sentence: {sentence}")
        tmp.append([])  # 문제가 있는 경우 빈 리스트를 추가

  0%|          | 0/34710 [00:00<?, ?it/s]

Error processing sentence:                                                
Error processing sentence:                                            
Error processing sentence:                                           
Error processing sentence:                                                
Error processing sentence:                                                
Error processing sentence:                                            
Error processing sentence:                                           
Error processing sentence:                                         
Error processing sentence:                                                                       
Error processing sentence:                                           


In [9]:
df["tokens"] = tmp

In [11]:
df["tokens"]

0           [수분, 크림, 피부, 발리, 라네즈, 크림]
1            [크림, 크림, 특유, 질감, 냄새, 크림]
2           [수분, 크림, 발림, 오랜만, 만족, 크림]
3           [발림, 흡수력, 부담, 크림, 수분, 크림]
4          [원지, 완벽주의, 원지, 완전, 로션, 화장]
                     ...             
34705               [가족, 선물, 트러블, 크림]
34706    [중인, 크림, 자극, 발림, 성도, 하루, 종일]
34707                        [결정, 색상]
34708       [마무리, 야외, 활동, 여름, 전체, 추천]
34709          [없어, 일주일, 동안, 자외선, 차단]
Name: tokens, Length: 34710, dtype: object

In [12]:
df.to_csv("C:/Users/LG/OneDrive - 숙명여자대학교/텍스트마이닝과자연어분석/suncream_review_token.csv", encoding = "utf-8", index = False)